<a href="https://colab.research.google.com/github/frh02/ProjectEEG/blob/master/vgg_select_eeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from keras.layers import Input, Lambda, Dense, Flatten,Reshape,Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import os


In [2]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/My Drive/Data_Combined/train'
valid_path = '/content/drive/My Drive/Data_Combined/validation'
test_path = '/content/drive/My Drive/Data_Combined/test'

vgg16_model = VGG16(weights="imagenet", include_top=True)

# (1) visualize layers
print("VGG16 model layers")
for i, layer in enumerate(vgg16_model.layers):
    print(i, layer.name, layer.output_shape)

# (2) remove the top layer
base_model = Model(input=vgg16_model.input, 
                   output=vgg16_model.get_layer("block5_pool").output)

# (3) attach a new top layer
base_out = base_model.output
base_out = Reshape((25088,))(base_out)
top_fc1 = Dropout(0.5)(base_out)
# output layer: (None, 5)
top_preds = Dense(3, activation="sigmoid")(top_fc1)

# (4) freeze weights until the last but one convolution layer (block4_pool)
for layer in base_model.layers[0:14]:
    layer.trainable = False

# (5) create new hybrid model
model = Model(input=base_model.input, output=top_preds)

# (6) compile and train the model
sgd = tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9)
model.compile(optimizer=sgd, loss="binary_crossentropy", metrics=["accuracy"])

553467904/553467096 [==============================] - 22s 0us/step
VGG16 model layers
0 input_1 (None, 224, 224, 3)
1 block1_conv1 (None, 224, 224, 64)
2 block1_conv2 (None, 224, 224, 64)
3 block1_pool (None, 112, 112, 64)
4 block2_conv1 (None, 112, 112, 128)
5 block2_conv2 (None, 112, 112, 128)
6 block2_pool (None, 56, 56, 128)
7 block3_conv1 (None, 56, 56, 256)
8 block3_conv2 (None, 56, 56, 256)
9 block3_conv3 (None, 56, 56, 256)
10 block3_pool (None, 28, 28, 256)
11 block4_conv1 (None, 28, 28, 512)
12 block4_conv2 (None, 28, 28, 512)
13 block4_conv3 (None, 28, 28, 512)
14 block4_pool (None, 14, 14, 512)
15 block5_conv1 (None, 14, 14, 512)
16 block5_conv2 (None, 14, 14, 512)
17 block5_conv3 (None, 14, 14, 512)
18 block5_pool (None, 7, 7, 512)
19 flatten (None, 25088)
20 fc1 (None, 4096)
21 fc2 (None, 4096)
22 predictions (None, 1000)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [3]:
# view the structure of the model
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [6]:
if not os.path.isdir('/content/drive/My Drive/Colab Notebooks/CNN_VGG_Weights'):
    os.mkdir('/content/drive/My Drive/Colab Notebooks/CNN_VGG_Weights')

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

valid_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale= 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Data_Combined/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

valid_set = valid_datagen.flow_from_directory('/content/drive/My Drive/Data_Combined/validation',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Data_Combined/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 7750 images belonging to 3 classes.
Found 2082 images belonging to 3 classes.
Found 1038 images belonging to 3 classes.


In [13]:
from keras.callbacks import ModelCheckpoint
#from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=100,
  steps_per_epoch=len(training_set),
  validation_steps=len(valid_set),
  callbacks=[ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/CNN_VGG_Weights/model_{epoch:03d}_{val_accuracy:.3f}.h5', 
                                            save_best_only=True,
                                            save_weights_only=False,
                                            monitor='val_accuracy')
          ],
initial_epoch=0
)

243/243 [==============================] - 248s 1s/step - loss: 0.5231 - accuracy: 0.7375 - val_loss: 0.4679 - val_accuracy: 0.7312
Epoch 29/100
243/243 [==============================] - 246s 1s/step - loss: 0.5207 - accuracy: 0.7406 - val_loss: 0.3951 - val_accuracy: 0.7425
Epoch 30/100
243/243 [==============================] - 247s 1s/step - loss: 0.5183 - accuracy: 0.7421 - val_loss: 0.5681 - val_accuracy: 0.7720
Epoch 31/100
243/243 [==============================] - 247s 1s/step - loss: 0.5139 - accuracy: 0.7434 - val_loss: 0.6518 - val_accuracy: 0.7720
Epoch 32/100
243/243 [==============================] - 247s 1s/step - loss: 0.5078 - accuracy: 0.7458 - val_loss: 0.4223 - val_accuracy: 0.7585
Epoch 33/100
243/243 [==============================] - 247s 1s/step - loss: 0.5097 - accuracy: 0.7484 - val_loss: 0.4451 - val_accuracy: 0.7823
Epoch 34/100
243/243 [==============================] - 248s 1s/step - loss: 0.5068 - accuracy: 0.7469 - val_loss: 0.4341 - val_accuracy: 0.755

In [ ]:
#model.load_weights(bestmodel)
model = keras.models.load_model('drive/My Drive/Colab Notebooks/model_VGG19_DATASET1/model_003_0.566')

In [ ]:
results = model.evaluate(training_set,steps=len(training_set))
print('Train accuracy:', results[1]*100)

In [ ]:
results = model.evaluate(test_set,steps=len(test_set))
print('Test accuracy:', results[1]*100)